In [1]:
import pdb
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

In [2]:
print("Begin Data Preparing")
pixels = ["img_{0}".format(i) for i in range(1600)]

Begin Data Preparing


In [3]:
def to_image(df):
    return np.expand_dims(df[pixels], axis=-1).reshape(-1, 40,40)

In [4]:

def to_image_2d(df):
    foo =  np.expand_dims(np.expand_dims(df[ ["img_{0}".format(i) for i in range(40*40)]], axis=-1).reshape(-1,40,40), axis=-1)        
    return foo

In [5]:
batch_size = 32

In [6]:
store_train = pd.HDFStore('../train_image.h5')
df_train_images = store_train.select("table",stop=1200000)
n_train_samples = int((store_train.get_storer('table').nrows/batch_size))*batch_size


store_val = pd.HDFStore('../val_image.h5')
df_val_images = store_val.select("table",stop=1200000)
n_val_samples = int((store_val.get_storer('table').nrows/batch_size))*batch_size

store_test = pd.HDFStore('../test_image.h5')
df_test_images = store_test.select("table",stop=1200000)
n_test_samples = int((store_test.get_storer('table').nrows/batch_size))*batch_size


In [7]:
df_train_images.head()

,img_0,img_1,img_2,img_3,img_4,img_5,img_6,img_7,img_8,img_9,...,img_1592,img_1593,img_1594,img_1595,img_1596,img_1597,img_1598,img_1599,is_signal_new,mass
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.394016
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.997341
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.648197
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.098465
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,174.350220


In [8]:
df_val_images.head()

,img_0,img_1,img_2,img_3,img_4,img_5,img_6,img_7,img_8,img_9,...,img_1592,img_1593,img_1594,img_1595,img_1596,img_1597,img_1598,img_1599,is_signal_new,mass
43221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.839817
77280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.690308
78615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192.865814
26383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.494675
789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.776985


In [9]:
def datagen_batch_h5(brs, store, batch_size=1024):
    """Generates data in batchaes using partial reading of h5 files """

    size = store.get_storer('table').nrows    
    i_start = 0
    step = 0
    while True:
        if size >= i_start+batch_size:            
            foo = store.select('table',
                               columns = brs,
                               start = i_start,
                               stop  = i_start + batch_size)
            yield foo
            i_start += batch_size
            step += 1
        else:
            size = store.get_storer('table').nrows
            i_start = 0

In [10]:
datagen_train = datagen_batch_h5('is_signal_new', store_train, batch_size = batch_size)
datagen_test  = datagen_batch_h5('is_signal_new', store_test, batch_size = batch_size)
datagen_val   = datagen_batch_h5('is_signal_new', store_val, batch_size = batch_size)

In [11]:
xx = next(datagen_train)

In [12]:
len(xx)

32

In [13]:
training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df_train_images[pixels].values.reshape(-1, 40, 40), tf.float32),
            tf.cast(df_train_images['is_signal_new'].values, tf.int32)
        )
    )
)

validation_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df_val_images[pixels].values.reshape(-1, 40, 40), tf.float32),
            tf.cast(df_val_images['is_signal_new'].values, tf.int32)
        )
    )
)

KeyError: "None of [Index(['img_0', 'img_1', 'img_2', 'img_3', 'img_4', 'img_5', 'img_6', 'img_7',\n       'img_8', 'img_9',\n       ...\n       'img_1590', 'img_1591', 'img_1592', 'img_1593', 'img_1594', 'img_1595',\n       'img_1596', 'img_1597', 'img_1598', 'img_1599'],\n      dtype='object', length=1600)] are in the [columns]"

In [19]:
n_constit = 2
feat_list =  ["E","PX","PY","PZ"]
cols = ["{0}_{1}".format(feature,constit)
        for feature in feat_list for constit in range(n_constit)]

In [20]:
cols

['E_0', 'E_1', 'PX_0', 'PX_1', 'PY_0', 'PY_1', 'PZ_0', 'PZ_1']

In [21]:
vec4 = np.expand_dims(df_train_images[cols],axis=-1).reshape(-1, len(feat_list), n_constit)

In [23]:
(-1, len(feat_list), n_constit)

(-1, 4, 2)

In [22]:
vec4

array([[[ 474.07114 ,  103.23624 ],
        [-250.34703 ,  -48.866222],
        [-223.65196 ,  -56.790775],
        [-334.7381  ,  -71.02549 ]],

       [[ 150.50453 ,   82.25706 ],
        [ 120.06239 ,   63.80174 ],
        [  76.852005,   42.754807],
        [ -48.274265,  -29.454842]],

       [[ 251.64539 ,  104.1478  ],
        [  10.427651,   10.718256],
        [-147.57375 ,  -54.497948],
        [ 203.56488 ,   88.101395]],

       ...,

       [[  64.15237 ,   50.308674],
        [  47.883705,   45.10902 ],
        [  17.668383,   20.591967],
        [  38.865215,    8.49174 ]],

       [[ 132.56255 ,   52.84212 ],
        [ 103.131134,   41.600754],
        [  32.62031 ,   13.222459],
        [ -76.63363 ,  -29.77975 ]],

       [[  83.333534,   80.80607 ],
        [  -9.745   ,   -8.434953],
        [ -82.62369 ,  -74.78946 ],
        [  -4.778968,  -29.411036]]], dtype=float32)

In [24]:
E     = vec4[:,0,:]
pxs   = vec4[:,1,:]
pys   = vec4[:,2,:]
pzs   = vec4[:,3,:]
pT    = np.sqrt(pxs**2+pys**2)

In [26]:
E.shape

(1200000, 2)

In [14]:
pixel_brs = []
pixel_brs += ["img_{0}".format(i) for i in range(40*40)]

In [16]:
len(pixel_brs)

1600

In [ ]:
df = df.iloc[np.random.permutation(len(df))]
X = datagen_train
y = tf.keras.utils.to_categorical(df[clf.params["signal_branch"]].values,clf.params["n_classes"])

In [9]:
def datagen_batch_h5(brs, store, batch_size=1024):
    """Generates data in batchaes using partial reading of h5 files """

    size = store.get_storer('table').nrows    
    i_start = 0
    step = 0
    while True:
        if size >= i_start+batch_size:            
            foo = store.select('table',
                               columns = brs,
                               start = i_start,
                               stop  = i_start + batch_size)
            yield foo
            i_start += batch_size
            step += 1
        else:
            size = store.get_storer('table').nrows
            i_start = 0

In [10]:
print("Total number of training samples = ", n_train_samples)
print("Total number of testing samples = ", n_test_samples)
print("Total number of valing samples = ", n_val_samples)
samples_per_epoch = n_train_samples
samples_per_epoch_test = n_test_samples
samples_per_epoch_val = n_val_samples

Total number of training samples =  1210976
Total number of testing samples =  402976
Total number of valing samples =  404000


In [11]:
n_classes = 2

In [12]:
def model_2d(params):
    activ = lambda : Activation('relu')
    model = models.Sequential()
    model.add(layers.Conv2D(32,(3,3),padding='same',data_format='channels_last', activation="relu",input_shape=(40, 40, 1)))
    model.add(layers.Conv2D(32,(3,3),padding='same',data_format='channels_last', activation="relu"))

    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(32,(3,3),padding='same',data_format='channels_last', activation="relu"))
    model.add(layers.Conv2D(32,(3,3),padding='same',data_format='channels_last', activation="relu"))

    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(32,(3,3),padding='same',data_format='channels_last', activation="relu"))
    model.add(layers.Conv2D(32,(3,3),padding='same',data_format='channels_last', activation="relu"))

    model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Flatten())
    
    model.add(layers.Dense(400,activation="relu"))
    model.add(layers.Dense(200,activation="relu"))
    model.add(layers.Dense(100,activation="relu"))
    model.add(layers.Dense(50,activation="relu"))
    model.add(layers.Dense(10,activation="relu"))

    model.add(layers.Dense(nclasses))
    model.add(Activation('softmax'))

    return model

In [13]:
class_names = {}
for i in range(2):
    class_names[i] = "c{0}".format(i)

In [14]:
class_names

{0: 'c0', 1: 'c1'}

In [ ]:
clf.model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [ ]:
                
train_gen = generator(clf.datagen_train)
val_gen  = generator(clf.datagen_val) # CORRECTED

early_stop = EarlyStopping(monitor='val_loss', 
                               patience=clf.params["early_stop_patience"], 
                               verbose=0, 
                               mode='auto')

In [ ]:
def train_keras(clf):

    print("Starting train_keras with the parameters: ")
    for k,v in clf.params.items():
        print("\t", k,"=",v)

    outdir = clf.params["output_path"] + clf.name

    if not os.path.exists(outdir):
        os.makedirs(outdir)
      
    # Prepare model and train
    #sgd = SGD(lr = clf.params["lr"], 
    #          decay = clf.params["decay"], 
    #          momentum = clf.params["momentum"], 
    #          nesterov=True)
    clf.model.compile(loss='categorical_crossentropy', 
                      optimizer="adam", 
                      metrics=["accuracy"])


    if clf.params["reweight_events"]:
        f = open("mass_weights.pickle", "rb")
        di = pickle.load(f)
        
        counts_qcd  = di["counts_qcd"]
        bins_qcd    = di["bins_qcd"]
        counts_top  = di["counts_top"]
        bins_top    = di["bins_top"]
       
        
        bins_qcd = bins_qcd[:-1]+(bins_qcd[1:]-bins_qcd[:-1])/2.
        bins_top = bins_top[:-1]+(bins_top[1:]-bins_top[:-1])/2.
     
        bins = [bins_qcd, bins_top]
        counts = [counts_qcd, counts_top]
        
        c_min =[min(counts[0][np.nonzero(counts[0])]), min(counts[1][np.nonzero(counts[1])])]

        def calc_w(m,s):

            if m>max(bins[s]) or m<min(bins[s]):
                return 1/c_min[s]

            c = counts[s][np.abs(m-bins[s]).argmin()]
            c = max(c_min[s],c)

            return 1/c


    print("Calling fit_generator")

    def generator(dg):
        while True:
            df = next(dg)
            
            # Shuffle
            df = df.iloc[np.random.permutation(len(df))]

            X = clf.image_fun(df)
            y = np_utils.to_categorical(df[clf.params["signal_branch"]].values,clf.params["n_classes"])

            if clf.params["reweight_events"]:
                
                Es = ["E_{0}".format(i) for i in range(clf.params["n_constit"])]
                PXs = ["PX_{0}".format(i) for i in range(clf.params["n_constit"])]
                PYs = ["PY_{0}".format(i) for i in range(clf.params["n_constit"])]
                PZs = ["PZ_{0}".format(i) for i in range(clf.params["n_constit"])]

                E_tot = df[Es].sum(axis=1)
                PX_tot = df[PXs].sum(axis=1)
                PY_tot = df[PYs].sum(axis=1)
                PZ_tot = df[PZs].sum(axis=1)

                M = np.sqrt(np.power(E_tot,2) - np.power(PX_tot,2) - np.power(PY_tot,2) - np.power(PZ_tot,2))
                w = np.array([calc_w(m,s) for m,s in zip(M,df["is_signal_new"])])
                        
                yield  X,y, w

            else:        
                yield X,y

                
    train_gen = generator(clf.datagen_train)
    val_gen  = generator(clf.datagen_val) # CORRECTED

    early_stop = EarlyStopping(monitor='val_loss', 
                               patience=clf.params["early_stop_patience"], 
                               verbose=0, 
                               mode='auto')

    filepath= outdir + "/weights-latest.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

    # save the architecture
    model_out_yaml = open(outdir + "/" + clf.name + ".yaml", "w")
    model_out_yaml.write(clf.model.to_yaml())
    model_out_yaml.close()

    print("Steps: Train: {0} Validation: {1}".format(int(clf.params["samples_per_epoch"]/clf.params["batch_size"]), # CORRECTED
                                               int(clf.params["samples_per_epoch_val"]/clf.params["batch_size"]))) # CORRECTED

    ret = clf.model.fit_generator(train_gen,
                                  steps_per_epoch = clf.params["samples_per_epoch"]/clf.params["batch_size"],
                                  validation_steps = clf.params["samples_per_epoch_val"]/clf.params["batch_size"], # CORRECTED
                                  #verbose=0, 
                                  epochs = clf.params["nb_epoch"],
                                  validation_data=val_gen, # CORRECTED
                                  callbacks = [checkpoint, early_stop, LossPlotter(outdir)])
    
    print("fit Done")

    plt.clf()
    plt.plot(ret.history["acc"])
    plt.plot(ret.history["val_acc"])
    plt.savefig(outdir + "/acc.png")

    plt.clf()
    plt.plot(ret.history["loss"])
    plt.plot(ret.history["val_loss"])
    plt.savefig(outdir + "/loss.png")

    #valacc_out = open(outdir + "/valacc.txt", "w")
    #valacc_out.write(str(ret.history["val_acc"][-1]) + "\n")
    #valacc_out.close()

    #maxvalacc_out = open(outdir + "/maxvalacc.txt", "w")
    #maxvalacc_out.write(str(max(ret.history["val_acc"])) + "\n")
    #maxvalacc_out.close()
  
    #deltaacc_out = open(outdir + "/" + clf.name + "deltaacc.txt", "w")
    #deltaacc_out.write(str(ret.history["val_acc"][-1] - ret.history["acc"][-1]) + "\n")
    #deltaacc_out.close()

    # save the architecture
    model_out_yaml = open(outdir + "/" + clf.name + ".yaml", "w")
    model_out_yaml.write(clf.model.to_yaml())
    model_out_yaml.close()
  
    # And the weights
    clf.model.save_weights(outdir + "/" + clf.name + '_weights.h5', overwrite=True)

In [ ]:
def prepare(self):
    if not self.load_from_file:
        if self.backend == "scikit":
            train_scikit(self)
        elif self.backend == "keras":
            return train_keras(self)
    else:
        if self.backend == "scikit":
            f = open(os.path.join(self.inpath,self.name + ".pickle"), "r")
            self.model = pickle.load(f)
            f.close()
        elif self.backend == "keras":
            print("Loading", self.name)
            f = open(os.path.join(self.inpath,self.name + ".yaml"), "r")
            yaml_string = f.read()
            f.close()       
            print("Getting yaml")
            self.model = model_from_yaml(yaml_string,  custom_objects={"LoLa":LoLa, "Convert":Convert})                
            print("Got yaml")
            self.model.load_weights(os.path.join(self.inpath,self.name + "_weights.h5"))       
            print("Loading", self.name, "from file: Done...")

In [ ]:
for i_batch in range(nbatches):
    df = next(clf.datagen_test) # CORRECTED
    if clf.backend == "keras":
        X = clf.image_fun(df)        
        probs = clf.model.predict_on_batch(X)
    else:        
        X = get_data_vars(df, clf.varlist)
        probs = clf.model.predict_proba(X)
 
        probs = np.nan_to_num(probs)

        # prediction returns two values: 
        # signal and background probability
        # we're just interested in the signal prob (bg prob = 1 - signal_prob)        
        df["sigprob_" + clf.name] = probs[:,1] 

        # Label is maximum classiier
        df["label_" + clf.name]   = np.apply_along_axis(lambda x:int(np.argmax(x)),1,probs)
                
        # Add per-class probs
        probnames= []
        for iclass in range(clf.params["n_classes"]):
            probname = "cprob_{0}_{1}".format(iclass, clf.name)            
            df[probname] = probs[:,iclass]
            probnames.append(probname)
            
        # Now that we have calculated the classifier response, 
        # remove the rest
        cols_to_keep = set(["entry", 
                            clf.params["signal_branch"], 
                            "label_" + clf.name,
                            "sigprob_" + clf.name] + probnames )
        cols_to_drop = list(set(df.columns) - cols_to_keep)
        df = df.drop(cols_to_drop,axis=1)

        df_all = df_all.append(df)

In [ ]:
nbatches = int(clf.params["samples_per_epoch_test"]/clf.params["batch_size"] - 1) # CORRECTED

    # Enable to print weights
    if False: 
        for layer in clf.model.layers:
            weights = layer.get_weights()
            print(weights)


    df_all = pandas.DataFrame()
    
    # Loop over batches
    for i_batch in range(nbatches):

        df = next(clf.datagen_test) # CORRECTED

        if clf.backend == "keras":
            X = clf.image_fun(df)        
            probs = clf.model.predict_on_batch(X)
        else:        
            X = get_data_vars(df, clf.varlist)
            probs = clf.model.predict_proba(X)
 
        probs = np.nan_to_num(probs)

        # prediction returns two values: 
        # signal and background probability
        # we're just interested in the signal prob (bg prob = 1 - signal_prob)        
        df["sigprob_" + clf.name] = probs[:,1] 

        # Label is maximum classiier
        df["label_" + clf.name]   = np.apply_along_axis(lambda x:int(np.argmax(x)),1,probs)
                
        # Add per-class probs
        probnames= []
        for iclass in range(clf.params["n_classes"]):
            probname = "cprob_{0}_{1}".format(iclass, clf.name)            
            df[probname] = probs[:,iclass]
            probnames.append(probname)
            
        # Now that we have calculated the classifier response, 
        # remove the rest
        cols_to_keep = set(["entry", 
                            clf.params["signal_branch"], 
                            "label_" + clf.name,
                            "sigprob_" + clf.name] + probnames )
        cols_to_drop = list(set(df.columns) - cols_to_keep)
        df = df.drop(cols_to_drop,axis=1)

        df_all = df_all.append(df)

    #pdb.set_trace()

    #ll = log_loss(df_all[clf.params["signal_branch"]], df_all[probnames])
    #print("Log loss: {0}".format(ll))

    cm = confusion_matrix(df_all[clf.params["signal_branch"]], df_all["label_" + clf.name])
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Confusion Matrix:")
    print(cm)

    # Area under curve
    AUC = roc_auc_score(df_all[clf.params["signal_branch"]], df_all["sigprob_" + clf.name])
    print("AUC: {0}".format(AUC))

    # Mutual Information
    mi = max([mutual_info_score(df_all["is_signal_new"], df_all["sigprob_" + clf.name]<thres) for thres in np.arange(0,1,0.01)])
    print("MI: {0}".format(mi))

    # Normalized Mutual Information
    nmi = max([normalized_mutual_info_score(df_all["is_signal_new"], df_all["sigprob_" + clf.name]<thres) for thres in np.arange(0,1,0.01)])
    print("NMI: {0}".format(nmi))
    
    fpr, tpr, _ = roc_curve(df_all[clf.params["signal_branch"]], df_all["sigprob_" + clf.name])

    outdir = clf.params["output_path"] + clf.name
    plt.clf()
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {0:.2f})'.format(AUC))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.savefig(outdir + "/roc.png")


    store_df = pandas.HDFStore('output_' + clf.name + suffix + '.h5')
    store_df["all"] = df_all
    store_df.close()
    
    return -1. * AUC

In [ ]:
        TCB.Classifier(params["model_name"],
                   "keras",
                   params,
                   params["read_from_file"],
                   datagen_train_pixel,                   
                   datagen_test_pixel,               
                   datagen_val_pixel,               
                   the_model(params),
                   image_fun = the_image_fun,           
                   class_names = class_names,
                   inpath = "Lola_Poly2/",
               )

In [ ]:
if params["inputs"] == "2d" or params["inputs"] == "caps":
    pixel_brs += ["img_{0}".format(i) for i in range(40*40)]
elif params["inputs"] == "constit_fcn":
    pixel_brs += ["{0}_{1}".format(feature,constit) for feature in feat_list for constit in range(params["n_constit"])]
elif params["inputs"] == "constit_lola":
    pixel_brs += ["{0}_{1}".format(feature,constit) for feature in feat_list for constit in range(params["n_constit"])]